<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/MultilayerPerceptronClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
import numpy as np

class MLP(tf.keras.Model):
  def __init__(self,layer_size):
    super().__init__()
    self.weights = []
    self.bias = []

    for in_dim, out_dim in zip(layer_size[:-1], layer_size[1:]):
      w = self.weight(shape = (in_dim, out_dim), initializer = tf.keras.initializers.HeNormal(),trainable = True)
      b = self.weight(shape=(out_dim), initializer = "zeros", trainable = True)

      self.weights.append(w)
      self.bias.append(b)

  def call(self, x):
    out = x
    for w,b in zip(self.weights[:-1], self.bias[:-1]):
      out = tf.nn.relu(tf.matmul(out, w)+b)
    logits = tf.matmul(out,self.weights[-1]+self.bias[-1])
    #for multiclass classification.
    return tf.nn.softmax(logits)
    # for binary classification
    # return tf.nn.sigmoid(logits)

num_samples = 1000
num_features = 32
num_classes = 10

X = np.random.randn(num_samples, num_features).astype('float32')
y = np.random.randint(0, num_classes, size=(num_samples,))

model = MLP([32, 128, 128, 64, 10])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


AttributeError: property 'weights' of 'MLP' object has no setter

In [16]:
import tensorflow as tf
import numpy as np

# -------------------------
# 1. Create dummy dataset
# -------------------------
# num_samples = 1000
# num_features = 32
# num_classes = 10

# X = np.random.randn(num_samples, num_features).astype("float32")
# y = np.random.randint(0, num_classes, size=(num_samples,))

# Load MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize to 0–1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten 28x28 → 784
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# -------------------------
# 2. Build your MLP model
# -------------------------
class MLP(tf.keras.Model):
    def __init__(self, layer_size):
        super().__init__()
        self.weights_list = []
        self.bias_list = []

        for in_dim, out_dim in zip(layer_size[:-1], layer_size[1:]):
            w = self.add_weight(
                shape=(in_dim, out_dim),
                initializer=tf.keras.initializers.HeNormal(),
                trainable=True
            )
            b = self.add_weight(
                shape=(out_dim,),
                initializer="zeros",
                trainable=True
            )
            self.weights_list.append(w)
            self.bias_list.append(b)

    def call(self, x):
        out = x
        for w, b in zip(self.weights_list[:-1], self.bias_list[:-1]):
            # out = tf.nn.relu(tf.matmul(out, w) + b)
            out = tf.nn.gelu(tf.matmul(out, w) + b)

        logits = tf.matmul(out, self.weights_list[-1]) + self.bias_list[-1]
        #for multiclass classification.
        return tf.nn.softmax(logits)
        # for binary classification
        # return tf.nn.sigmoid(logits)

# -------------------------
# 3. Instantiate model
# -------------------------
model = MLP([32, 512, 512, 256, 128, 64, 10])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# -------------------------
# 4. Train model
# -------------------------
model.fit(X, y, epochs=5, batch_size=32)

# -------------------------
# 5. Evaluate model
# -------------------------
loss, acc = model.evaluate(X, y)
print("Test accuracy:", acc)

# -------------------------
# 6. Make predictions
# -------------------------
sample = X[:5]
pred = model(sample)
print("Predicted class probabilities:\n", pred.numpy())
print("Predicted classes:", tf.argmax(pred, axis=1).numpy())

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0913 - loss: 2.4876
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3585 - loss: 1.9984
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6158 - loss: 1.4391
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8382 - loss: 0.7739
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9453 - loss: 0.3803
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9949 - loss: 0.1136
Test accuracy: 0.9950000047683716
Predicted class probabilities:
 [[9.5197774e-06 5.4981861e-02 2.7229890e-04 1.4732249e-02 1.4558405e-03
  2.7292026e-03 7.6400829e-03 2.0488054e-03 4.6069410e-02 8.7006074e-01]
 [1.2608442e-03 1.0285276e-02 6.9765034e-03 4.5626857e-03 9.6780034e-03
  4.8830556e-03 9.1468716e-01 1.6435470e-02 2.5295625e-02 5.9353942e-03]
 [2.1668662e-04 7.1091548e-02 2.8192606e-03 8.9289612e-01 7.9095870e-04
  1.1794876e-03 2.7606606e-03 4.6728649e-03 1.2175925e-02 1.1396542e-02]
 [4.38